In [1]:
import Snapshot_code
import pandas as pd

new_df2 = pd.read_excel(r"C:\Users\NahianSiddique\OneDrive - Blend 360\Documents\GitHub\Snapshot2.0\appended_file_21_22_20221207 - Copy.xlsx")
# new_df2 = pd.read_excel(r"C:\Users\NahianSiddique\OneDrive - Blend 360\Hilton\Analytical Projects\HGV 2022 VIP Analysis\Data\Model Sample\appended_file_21_22_20221207.xlsx") - full data

mapping_dict = {"Baseline": "dataset_1", "Segment_1": "dataset_2", "Segment_2": "dataset_3"}

# Set up variables for snapshot
file_name = "profiles"
seg_var = "source"
bin_vars_path = "Data/Epsilon_attributes_binning_2.csv"
# Read in file and set bins



In [3]:
new_df2['source'] = new_df2['source'].map({1:'dataset_1',2:'dataset_2',3:'dataset_3'})

In [4]:
new_df2

,daydiff_since_first_pkg,daydiff_since_last_pkg,prior_all_pkgs,prior_3y_pkgs,prior_1y_pkgs,sum_all_pkg_price,sum_3y_pkg_price,sum_1y_pkg_price,daydiff_since_first_tour,daydiff_since_last_tour,...,slide_any,vids_HGV_duration,vids_ELARA_duration,link_HGV_club_duration,link_any_duration,slide_HCC_duration,slide_unknown_duration,slide_resort_photo_collection_duration,slide_any_duration,source
0,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_1
1,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_1
2,2758.0,1833.0,2,0,0,644.34,NaN,NaN,2269.0,1625.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_1
3,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_1
4,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96393,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_3
96394,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_3
96395,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_3
96396,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset_3


In [ ]:
profile = Snapshot_code.Snapshot_Profile(
    profile_data=new_df2,
    segment_var=seg_var,
    continuous_path=bin_vars_path,
    segments=None,
    segment_names=None,
    include=None,
    variable_order=None,
    other_segment=False,
    file=file_name,
    exclude=[],
    PPT=False,
    continuous=[],
    excludeother=False,
    mapping_dict=mapping_dict,
)
